In [11]:
from attrdict import AttrDict
try:
	import cPickle as pickle
except ImportError:
	import pickle
    
from src.dataloader import *
from src.model_dual import *
from src.model_target import *
from src.train import *
from src.train_target import *
    
device = 'cuda:0'

In [12]:
with open('./models/EMNLP_Final__12_6_enc_dec_layers_interleave_co_self_reascan__learned_pos_embed__lr_0.0001__bs_32__epochs_100__gpu_0__3420/config.p', 'rb') as f:
    config = AttrDict(pickle.load(f))
    
with open('./models/EMNLP_Final__12_6_enc_dec_layers_interleave_co_self_reascan__learned_pos_embed__lr_0.0001__bs_32__epochs_100__gpu_0__3420/config_target_predictor.p', 'rb') as f:
    model_config = AttrDict(pickle.load(f))

In [13]:
config.include_target = False

In [14]:
command_lang = torch.load('./models/EMNLP_Final__12_6_enc_dec_layers_interleave_co_self_reascan__learned_pos_embed__lr_0.0001__bs_32__epochs_100__gpu_0__3420/command_lang.pth')
action_lang = torch.load('./models/EMNLP_Final__12_6_enc_dec_layers_interleave_co_self_reascan__learned_pos_embed__lr_0.0001__bs_32__epochs_100__gpu_0__3420/action_lang.pth')

In [15]:
model = MultiModalModel_Dual(config).to(device)

checkpoint = torch.load('./models/EMNLP_Final__12_6_enc_dec_layers_interleave_co_self_reascan__learned_pos_embed__lr_0.0001__bs_32__epochs_100__gpu_0__3420/model.pt', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [16]:
target_predictor = Target_Predictor(model, model_config).to(device)

checkpoint = torch.load('./models/EMNLP_Final__12_6_enc_dec_layers_interleave_co_self_reascan__learned_pos_embed__lr_0.0001__bs_32__epochs_100__gpu_0__3420/target_predictor.pt', map_location=device)
target_predictor.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
test_data_paths = ['./data/ReaSCAN-v1.1/ReaSCAN-compositional-a1/test__and_replaced.json', 
                   './data/ReaSCAN-v1.1/ReaSCAN-compositional-a2/test__and_replaced.json', 
                   './data/ReaSCAN-v1.1/ReaSCAN-compositional-a3/test__and_replaced.json', 
                   './data/ReaSCAN-v1.1/ReaSCAN-compositional-b1/test__and_replaced.json', 
                   './data/ReaSCAN-v1.1/ReaSCAN-compositional-b2/test__and_replaced.json', 
                   './data/ReaSCAN-v1.1/ReaSCAN-compositional-c1/test__and_replaced.json', 
                   './data/ReaSCAN-v1.1/ReaSCAN-compositional-c2/test__and_replaced.json', 
                   './data/custom/new_c1_c2_splits/new_c1__and_replaced.json', 
                   './data/custom/new_c1_c2_splits/new_c2__and_replaced.json']

action_seq_errors_all_splits__and_replaced = []
target_errors_all_splits__and_replaced = []

for index, test_path in enumerate(test_data_paths):
    test_dataloader = dataloader(test_path, device, batch_size=config.batch_size, random_shuffle=False)
    
    action_seq_errors = []
    target_errors = []
    
    test_acc_action, action_seq_errors = validate_one_epoch(model, test_dataloader, command_lang, 
                                  action_lang, device, config, logger=None, exact_match=True, errors=action_seq_errors)
    
    _, test_acc_target, target_errors = validate_one_epoch_target(target_predictor, test_dataloader, command_lang, 
                                            action_lang, device, config, logger=None, errors=target_errors)
    
    print(test_path)
    print(test_acc_action)
    print(test_acc_target)
    
    action_seq_errors_all_splits__and_replaced.append(action_seq_errors)
    target_errors_all_splits__and_replaced.append(target_errors)

## Alternate Hypothesis Analysis

In [8]:
test_data_paths = ['./data/ReaSCAN-v1.1/ReaSCAN-compositional-c2/test.json']

for index, test_path in enumerate(test_data_paths):
    test_dataloader = dataloader(test_path, device, batch_size=config.batch_size, random_shuffle=False)
    
    test_acc_action, _ = validate_one_epoch(model, test_dataloader, command_lang, 
                                  action_lang, device, config, logger=None, exact_match=True)
    
    print(test_path)
    print(test_acc_action)

251it [01:52,  2.23it/s]

./data/ReaSCAN-v1.1/ReaSCAN-compositional-c2/test.json
0.30826693227091634


In [9]:
test_data_paths = ['./data/ReaSCAN-v1.1/ReaSCAN-compositional-c2/test__that_is_replaced.json']

for index, test_path in enumerate(test_data_paths):
    test_dataloader = dataloader(test_path, device, batch_size=config.batch_size, random_shuffle=False)
    
    test_acc_action, _ = validate_one_epoch(model, test_dataloader, command_lang, 
                                  action_lang, device, config, logger=None, exact_match=True)
    
    print(test_path)
    print(test_acc_action)

251it [01:53,  2.21it/s]

./data/ReaSCAN-v1.1/ReaSCAN-compositional-c2/test__that_is_replaced.json
0.3062749003984064


In [17]:
test_data_paths = ['./data/ReaSCAN-v1.1/ReaSCAN-compositional-c2/test__gt_that_is_replaced.json']

for index, test_path in enumerate(test_data_paths):
    test_dataloader = dataloader(test_path, device, batch_size=config.batch_size, random_shuffle=False)
    
    test_acc_action, _ = validate_one_epoch(model, test_dataloader, command_lang, 
                                  action_lang, device, config, logger=None, exact_match=True)
    
    print(test_path)
    print(test_acc_action)

117it [00:50,  2.29it/s]

./data/ReaSCAN-v1.1/ReaSCAN-compositional-c2/test__gt_that_is_replaced.json
0.1573648272017837


In [18]:
test_data_paths = ['./data/ReaSCAN-v1.1/ReaSCAN-compositional-c2/test__gt_command_that_is_replaced.json']

for index, test_path in enumerate(test_data_paths):
    test_dataloader = dataloader(test_path, device, batch_size=config.batch_size, random_shuffle=False)
    
    test_acc_action, _ = validate_one_epoch(model, test_dataloader, command_lang, 
                                  action_lang, device, config, logger=None, exact_match=True)
    
    print(test_path)
    print(test_acc_action)

117it [00:51,  2.29it/s]

./data/ReaSCAN-v1.1/ReaSCAN-compositional-c2/test__gt_command_that_is_replaced.json
0.15335841694537347
